# Transformers 101

This notebook serves as an exploration of the transformer architecture (Vaswani et. al.) Here, we'll implement in native PyTorch the basic building blocks of the transformer and then put them all together so we have a model architecture to put into `../models`

In the process of putting this together (much like my other exploratory projects) I tried to limit viewing existing code online, and primarily used my notes (pdf attached for anyone interested) as a foundation for this work.

PS: The notes in this notebook are very much stream of thought, interpret accordingly

In [1]:
import torch
import math
import torch.nn as nn
from torch.utils.data import random_split
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import AG_NEWS
from torch.utils.data import DataLoader

## Encoder

In the first half of the notebook we'll develop all the pieces needed to implement the transformer encoder, then train and test it on some task. 

We want something with output dims: (sequence_length, output_dim)

In [2]:
def positional_embedding(input_tensor: torch.Tensor, output_dim: int, n=10000): 
    """
    Here, we implement the naive approach from the original 
    paper with the sin and cosine functions. 
    """
    P = torch.zeros((input_tensor.shape[-1], output_dim))
    indices = torch.arange(input_tensor.size(-1))
    i_values = torch.arange(int(output_dim/2))
    denominators = torch.float_power(n, 2*i_values/output_dim)
    P[:, 0::2] = torch.sin(indices.unsqueeze(1) / denominators.unsqueeze(0)) # start at 0, step by 2 sin for even nums
    P[:, 1::2] = torch.cos(indices.unsqueeze(1) / denominators.unsqueeze(0)) # start at 1, step by 2 cos for odd nums
    return P


In [3]:
a = torch.rand((2, 5))
output_dims = 3
positional_embedding(a, output_dims)

tensor([[ 0.0000,  1.0000,  0.0000],
        [ 0.8415,  0.5403,  0.8415],
        [ 0.9093, -0.4161,  0.9093],
        [ 0.1411, -0.9900,  0.1411],
        [-0.7568, -0.6536, -0.7568]])

In [4]:
def attention(x): 
    """
    Simple dot product based attention
    """
    query_layer, key_layer, value_layer = nn.Linear(x.shape[-1], x.shape[-1]), nn.Linear(x.shape[-1], x.shape[-1]), nn.Linear(x.shape[-1], x.shape[-1])
    query, key, value = query_layer(x), key_layer(x), value_layer(x)
    attention_weights  = torch.nn.Softmax(-1)(torch.tensordot(query, key, dims=1))
    return torch.sum(value * attention_weights)

In [5]:
x = torch.rand(1, 12)
attention(x)

tensor(0.1738, grad_fn=<SumBackward0>)

Just to emulate how it would be implemented, we write out the add norm function below. However in practice, this will be encompassed by each transformer sub module since each of them are followed by addition with residual and layer normalization. 

In [6]:
def add_norm(residual: torch.Tensor, hidden: torch.Tensor): 
    if residual.shape != hidden.shape: 
        raise ValueError("Shapes mismatch")
    else: 
        output = residual + hidden # element wise addition
        layer_norm = nn.LayerNorm([residual.shape[-2], residual.shape[-1]])
        return layer_norm(output)

In [7]:
# usage example: 

tensor_a = torch.rand([1, 5, 6]) # batch size, sequence length, embedding dimensions
tensor_b = torch.rand([1, 5, 6])
print(tensor_a)
print(tensor_b)
print(f"Final: {add_norm(tensor_a, tensor_b)}")

tensor([[[0.7833, 0.9884, 0.5462, 0.3524, 0.5592, 0.4115],
         [0.1378, 0.6331, 0.4987, 0.1650, 0.2863, 0.0467],
         [0.6652, 0.0613, 0.9855, 0.1929, 0.7510, 0.1926],
         [0.1679, 0.4150, 0.0477, 0.0384, 0.9646, 0.1112],
         [0.9000, 0.0346, 0.9401, 0.6464, 0.6922, 0.9834]]])
tensor([[[0.7859, 0.7694, 0.0275, 0.7968, 0.8284, 0.9946],
         [0.6791, 0.0083, 0.4875, 0.6828, 0.9286, 0.4735],
         [0.6746, 0.5864, 0.0128, 0.3205, 0.0690, 0.1453],
         [0.1133, 0.6379, 0.8655, 0.2424, 0.0102, 0.9089],
         [0.6790, 0.6481, 0.8799, 0.7385, 0.6040, 0.0767]]])
Final: tensor([[[ 1.3746,  1.8270, -1.0120,  0.3677,  0.9394,  0.9837],
         [-0.4288, -0.8495, -0.0230, -0.3549,  0.5254, -1.1403],
         [ 0.8246, -0.8344,  0.0060, -1.1566, -0.4215, -1.5772],
         [-1.7133,  0.1368, -0.1981, -1.7144, -0.0502,  0.0582],
         [ 1.3983, -0.7508,  1.9759,  0.9328,  0.7201,  0.1543]]],
       grad_fn=<NativeLayerNormBackward0>)


In [8]:
def scaled_dot_product_attention(q, k, d_k, mask = None):
    scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        # think of the last two dimensions of the attention tensor to be attention values for token-token pairs based on their indices (like a look up table of attention weights)
        # the diagonal will be the attention to the current token (self). The upper diagonal will contain attention with respect to tokens in the future and the lower in the past
        # therefore our 
        scores = scores.masked_fill(mask == 0, float('-inf'))
    return nn.Softmax(-1)(scores)

d_k and d_v are essentially hyperparameters that are fixed before training. This allows for the query and keys to have the same dimensionality, and for all 3 of them to have consistent dimensionality. In many transformer implementations, d_k and d_v are set to be the same for simplicity but this is not always the case.

In [9]:
def multihead_attention(k, q, v, d_k, d_v, d_model, num_heads):
    """
    Scaled Dot product based multi-head attention
    """
    # declare projection layers - assume all inputs have d_model size in the last dimension, and project to number of heads * d_k or d_v 
    query_layer, key_layer, value_layer = nn.Linear(d_model, num_heads* d_k), nn.Linear(d_model, num_heads* d_k), nn.Linear(d_model, num_heads*d_v)
    k_len, q_len, v_len, batch_size = k.size(1), q.size(1), v.size(1),  q.size(0)
    residual = q

    # in the following line we apply the linear projections and then reshape the outputs for multihead attention. 
    #The reshaping splits the last dimension of the linear layer's output into num_heads and d_k (or d_v for value). 
    # This creates multiple "heads" in the tensor, each with its own d_k (or d_v) dimension
    k, q, v = key_layer(k).view(batch_size, k_len,  num_heads, d_k), query_layer(q).view(batch_size, q_len,  num_heads, d_k), value_layer(v).view(batch_size, v_len,  num_heads, d_v)
    
    # we perform the following transpose so that the num heads dimension preceeds the seq length dimension. This way, each head can capture different information about the same sequence
    q, k, v = q.transpose(1, 2), k.transpose(1, 2), v.transpose(1, 2)
    attention = scaled_dot_product_attention(q, k, d_k)
    print(f"attention shape: {attention.shape}, value_shape: {v.shape}")
    output = torch.matmul(attention, v)
    print(output.shape)
    # following reshaping is done so that we can add our output to the residual 
    output = output.transpose(1, 2).contiguous().view(batch_size, q_len, -1)
    concatenated_projection = nn.Linear(num_heads * d_v, d_model, bias=False)

    output = concatenated_projection(output)
    output += residual

    print(residual.shape)
    layer_norm = nn.LayerNorm([residual.shape[-2], residual.shape[-1]])
    output = layer_norm(output)

    return output, attention

In [10]:
d_model = 512

# from the paper: To facilitate these residual connections, all sub-layers in the model, as well as the embedding layers, produce outputs of dimension d model 
k, q, v = torch.rand((1, 2, d_model)), torch.rand((1, 2, d_model)), torch.rand((1, 2, d_model))
d_k, d_v = 5, 5
num_heads = 4
print(v.shape)
out, attn = multihead_attention(k, q, v, d_k, d_v, d_model, num_heads)
print(out.shape, attn.shape)

torch.Size([1, 2, 512])
attention shape: torch.Size([1, 4, 2, 2]), value_shape: torch.Size([1, 4, 2, 5])
torch.Size([1, 4, 2, 5])
torch.Size([1, 2, 512])
torch.Size([1, 2, 512]) torch.Size([1, 4, 2, 2])


In [11]:
class PositionWiseFFN(nn.Module): 
    def __init__(self, d_model, d_ff, dropout) -> None:
        super(PositionWiseFFN, self).__init__()
        self.fc1 = nn.Sequential(nn.Linear(d_model, d_ff, bias=True),nn.ReLU())
        self.fc2 = nn.Linear(d_ff, d_model, bias=True)
        self.layer_norm = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        residual = x
        x = self.fc2(self.fc1(x))        
        return self.dropout(self.layer_norm(x+residual))


In [12]:
ffn = PositionWiseFFN(d_model, 2048, 0.1)
x = torch.rand((1, 2, d_model))
ffn(x)

tensor([[[-0.0000e+00, -1.9143e+00, -1.7751e-01,  ..., -7.1614e-01,
           1.7585e+00, -2.4166e-02],
         [-7.9474e-02, -1.4749e-03, -1.4262e+00,  ...,  9.2286e-01,
           1.7577e+00, -9.9777e-02]]], grad_fn=<MulBackward0>)

Now that we've implemented the lowest level building blocks of the transormer, below we put them together to build transformer blocks, encoder and decoder layers, and the complete transformer architecture. Now we try and condense everything to a more concise-less experimental implementation. 

In [13]:
class MultiHeadAttention(nn.Module): 
    def __init__(self, d_k, d_model, d_v, dropout, num_heads) -> None:
        super(MultiHeadAttention, self).__init__()
        self.d_k, self.d_v, self.d_model, self.num_heads = d_k, d_v, d_model, num_heads
        self.query_layer, self.key_layer, self.value_layer = nn.Linear(d_model, num_heads* d_k), nn.Linear(d_model, num_heads* d_k), nn.Linear(d_model, num_heads*d_v)
        self.layer_norm = nn.LayerNorm(d_model)
        self.concat_projection = nn.Linear(num_heads*d_v, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, q, k, v, mask = None):
        k_len, q_len, v_len, batch_size = k.size(1), q.size(1), v.size(1),  q.size(0)
        residual = q
        k, q, v = self.key_layer(k).view(batch_size, k_len,  self.num_heads, self.d_k), self.query_layer(q).view(batch_size, q_len,  self.num_heads, self.d_k), self.value_layer(v).view(batch_size, v_len,  self.num_heads, self.d_v)
        q, k, v = q.transpose(1, 2), k.transpose(1, 2), v.transpose(1, 2)
        attention = scaled_dot_product_attention(q, k, self.d_k, mask)
        output = torch.matmul(attention, v)
        output = self.concat_projection(output.transpose(1, 2).contiguous().view(batch_size, q_len, -1))
        return self.dropout(self.layer_norm(output+residual))

In [14]:
class EncoderLayer(nn.Module): 
    def __init__(self, d_k, d_model, d_v, num_heads, d_ff, dropout) -> None:
        super(EncoderLayer, self).__init__()
        self.k_layer, self.q_layer, self.v_layer = nn.Linear(d_model, d_model), nn.Linear(d_model, d_model), nn.Linear(d_model, d_model)
        self.multihead_attention = MultiHeadAttention(d_k, d_model, d_v, dropout, num_heads)
        self.pointwise_ffn = PositionWiseFFN(d_model, d_ff, dropout)
    
    def forward(self, x): 
        k, q, v = self.k_layer(x), self.q_layer(x), self.v_layer(x)
        output = self.multihead_attention(q, k, v)
        return self.pointwise_ffn(output)

The following encoder implementation is based off of the block diagram from Attention Is All You Need

In [15]:
class Encoder(nn.Module):
    def __init__(self, d_k, d_model, d_v, d_ff, num_heads, num_layers, vocab_size, dropout=0.1) -> None:
        super(Encoder, self).__init__()
        self.d_model = d_model
        self.embedding = nn.Embedding(vocab_size, d_model, padding_idx=0)
        self.positional_embedding = positional_embedding
        self.dropout = nn.Dropout(dropout)
        self.layers = [EncoderLayer(d_k, d_model, d_v, num_heads, d_ff, dropout) for _ in range(num_layers)]
    
    def forward(self, x):
        embedded = self.embedding(x)
        x = self.dropout(embedded + self.positional_embedding(x, self.d_model).to(x.device))
        for layer in self.layers:
            layer = layer.to(x.device)
            x = layer(x)
        return x

## Preliminary testing on text classification task (encoder only)

In [16]:
class ClassificationTransformer(nn.Module): 
    def __init__(self, d_k, d_model, d_v, d_ff, num_heads, num_layers, num_classes, vocab_size, dropout=0.1) -> None:
        super(ClassificationTransformer, self).__init__()
        self.encoder_only_transformer = Encoder(d_k, d_model, d_v, d_ff, num_heads, num_layers, vocab_size, dropout=0.1)
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(d_model, d_model)
        self.fc2 = nn.Linear(d_model, num_classes)
    
    def forward(self, x):
        out = self.encoder_only_transformer(x)
        avg_pool = torch.mean(out, dim=-2)
        return self.fc2(self.dropout(self.fc1(self.dropout(avg_pool))))

In [17]:
train_iter = AG_NEWS(split='train')

# Convert to list to enable random splitting
train_dataset = list(train_iter)

#80-20 train-val split 
train_size = int(len(train_dataset) * 0.8)  
val_size = len(train_dataset) - train_size  
train_data, val_data = random_split(train_dataset, [train_size, val_size])

tokenizer = get_tokenizer("basic_english")

def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)

VOCAB_SIZE = 5000

# Build vocab based on the train_data
train_data_iter = (text for _, text in train_data)
vocab = build_vocab_from_iterator(yield_tokens(train_data_iter), specials=["<unk>"], max_tokens=VOCAB_SIZE)
vocab.set_default_index(vocab["<unk>"])

In [18]:
from torch.nn.utils.rnn import pad_sequence

text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1

def collate_batch(batch):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    label_list, text_list, lengths = [], [], []
    
    # Sort the batch in the descending order
    batch.sort(key=lambda x: len(x[1]), reverse=True)
    
    for _label, _text in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        lengths.append(processed_text.size(0))
        
    label_list = torch.tensor(label_list, dtype=torch.int64)
    lengths = torch.tensor(lengths, dtype=torch.int64)
    
    # Pad sequences
    text_list = pad_sequence(text_list, batch_first=True)
    
    return label_list.to(device), text_list.to(device), lengths

In [19]:
train_loader = DataLoader(train_data, batch_size = 8, shuffle = True, collate_fn = collate_batch)
val_loader = DataLoader(val_data, batch_size = 8, shuffle = False, collate_fn = collate_batch)

In [20]:
LEARNING_RATE = 5e-4
NUM_EPOCHS = 50
DROPOUT = 0.5
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

D_K = 128
D_V = 128
D_FF = 512
D_MODEL = 256
NUM_LAYERS = 2
OUTPUT_DIM = 4

In [21]:
model = ClassificationTransformer(D_K, D_MODEL, D_V, D_FF, num_heads=3, num_classes=OUTPUT_DIM, num_layers=2, vocab_size=VOCAB_SIZE)
model = model.to(DEVICE)

In [22]:
def train(model, train_loader, val_loader, loss_function, optim, epochs, device):
    losses = [] #group losses for loss visualization 
    running_loss = 0.0
    val_losses = []
    for epoch in range(epochs):
        model.train()
        print("Epoch %d / %d" % (epoch+1, epochs))
        print("-"*10)
    
        for i, batch_data in enumerate(train_loader):
            (y, x, x_size) = batch_data
            logits = model(x)
            loss = loss_function(logits, y)
            optim.zero_grad()
            loss.backward()
            optim.step()
            running_loss += loss.item()
            losses.append(loss)

            if (i+1) % 1000 == 0:
                print("Step: {}, average training loss over last 2000 steps: {:.4f}".format(i+1, running_loss/1000))
                running_loss = 0.0
            
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            correct_pred = 0.0
            num_samples = 0
            for i, batch_data in enumerate(val_loader):
                (y, x, x_size) = batch_data
                y, x, x_size = y.to(device), x.to(device), x_size.to(device)
                logits = model(x)
                loss = loss_function(logits, y)
                _, predicted_labels = torch.max(logits, 1)
                correct_pred += (predicted_labels.long() == y.long()).sum()
                num_samples+=predicted_labels.shape[0]
                val_loss += loss.item()
            
            val_accuracy = (correct_pred / num_samples) * 100
            val_losses.append(val_loss)
        print("Epoch: {}, validation loss: {:.4f}, val accuracy: {:.2f}".format(epoch+1, val_loss/len(val_loader), val_accuracy))
    
    return losses, val_losses

In [23]:
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

/home/aryaman.pandya/ml_accel/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
train_loss, val_loss = train(model, train_loader, val_loader, torch.nn.functional.cross_entropy, optimizer, NUM_EPOCHS, DEVICE)

Epoch 1 / 50
----------
Step: 1000, average training loss over last 2000 steps: 1.2368
Step: 2000, average training loss over last 2000 steps: 0.8032
Step: 3000, average training loss over last 2000 steps: 0.6700
Step: 4000, average training loss over last 2000 steps: 0.5944
Step: 5000, average training loss over last 2000 steps: 0.5584
Step: 6000, average training loss over last 2000 steps: 0.5351
Step: 7000, average training loss over last 2000 steps: 0.4960
Step: 8000, average training loss over last 2000 steps: 0.4939
Step: 9000, average training loss over last 2000 steps: 0.4759
Step: 10000, average training loss over last 2000 steps: 0.4260
Step: 11000, average training loss over last 2000 steps: 0.4329
Step: 12000, average training loss over last 2000 steps: 0.4271
Epoch: 1, validation loss: 0.3704, val accuracy: 87.67
Epoch 2 / 50
----------
Step: 1000, average training loss over last 2000 steps: 0.4143
Step: 2000, average training loss over last 2000 steps: 0.3966
Step: 3000, 

What do I want to see from this training? 

- I want to see how long it takes for this to converge 
- I want to see the computational complexity (how long per epoch?)
    - anywhere from 60 seconds an epoch to 3 minutes an epoch depending on hyperparameters 
    - With the low
- How does the model accuracy respond to changes in hyperparams? Not just LR and the usuals, but also d_model, d_k, d_v, d_ff
    - This is intuitive, higher values for these hyper params work better (until a point of diminishing return) since there's a balance between trying to capture all the intricacies of the data in high dimensional vector space and the actual complexity of the data to begin with. (as well as computational constraints) - doubling these dimensions led to more batch efficient learning but at the same time increased runtime by a proportional 2x 
    - 

Thoughts so far: 

This architecture is way more compute intensive but at the same time so much more batch efficient than previously explored ones when tuned with the right hyperparameters (LSTMs, RNNs even when bidirectional + multi layered and so on) that we see near 90% validation accuracy after just one or two epochs. These epochs, however, take tremendously long (7 minutes per epoch v/s the 1 minute LSTMs were taking). Part of this could be becauase of the implementation being from scratch (torch.transformer might be faster), but most of this comes from the computational load the multiple blocks (and their individual complexities)

Now that we have the encoder and were able to build something useful off of it and ensure its functionality, we should tweak our multihead attention to enable some form of masking and then build a decoder.

## Decoder

Now, we build the decoder and train and test it for some task

In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self, d_k, d_model, d_v, num_heads, d_ff, dropout) -> None:
        super(DecoderLayer, self).__init__()
        self.k_layer, self.q_layer, self.v_layer = nn.Linear(d_model, d_model), nn.Linear(d_model, d_model), nn.Linear(d_model, d_model)
        self.k_layer2, self.q_layer2, self.v_layer2 = nn.Linear(d_model, d_model), nn.Linear(d_model, d_model), nn.Linear(d_model, d_model)
        self.masked_multihead_attention = MultiHeadAttention(d_k, d_model, d_v, dropout, num_heads)
        self.multihead_attention = MultiHeadAttention(d_k, d_model, d_v, dropout, num_heads)
        self.positiontwise_ffn = PositionWiseFFN(d_model, d_ff, dropout)
    
    def forward(self, outputs, encoder_output):
        k1, q1, v1 = self.k_layer(outputs), self.q_layer(outputs), self.v_layer(outputs)
        outputs = self.masked_multihead_attention(q1, k1, v1, True)
        k2, q2, v2 = self.k_layer2(encoder_output), self.q_layer2(outputs), self.v_layer2(encoder_output)  # query comes from outputs of previous decoder layer while k, v come from the encoder's output
        outputs = self.multihead_attention(q2, k2, v2)
        return self.positiontwise_ffn(outputs)

In [ ]:
class Decoder(nn.Module):
    def __init__(self, d_k, d_model, d_v, d_ff, num_heads, num_layers, vocab_size, out_dims, dropout=0.1) -> None:
        super(Decoder, self).__init__()
        self.d_model = d_model
        self.embedding = nn.Embedding(vocab_size, d_model, padding_idx=0)
        self.positional_embedding = positional_embedding
        self.dropout = nn.Dropout(dropout)
        self.layers = [DecoderLayer(d_k, d_model, d_v, num_heads, d_ff, dropout) for _ in range(num_layers)]
        self.fc = nn.Linear(d_model, out_dims)
    
    def forward(self, x, encoder_output):
        embedded = self.embedding(x)
        x = self.dropout(embedded + self.positional_embedding(x, self.d_model).to(x.device))
        for layer in self.layers:
            layer = layer.to(x.device)
            x = layer(x, encoder_output)
        return self.fc(self.dropout(x))